In [ ]:
!pip install pytube
!pip install pydub
!pip install fastaudio
!pip install -U fastai

!pip install voila
!jupyter serverextension enable --sys-prefix voila

In [1]:
from pytube import YouTube
from moviepy import editor as mp
from pydub import AudioSegment
import wave, os
from fastai.vision.widgets import *
from fastai.vision.all import *
from fastaudio.core.all import *
from fastaudio.augment.all import *

In [4]:
# Define parameters
#from google.colab import drive
#drive.mount('/content/drive')

LENGTH_OF_CLIP = 5000
BASE_PATH = ""
if len(BASE_PATH) > 0 and BASE_PATH[-1] != '/': BASE_PATH += '/'

def _get_labels(x): return 1

In [5]:
def download_and_split(path, link):
    title = download(link, path)
    split_mp4_into_wavs(path + "test")
    return title

In [6]:
def split_mp4_into_wavs(path, length=LENGTH_OF_CLIP):
    filename = path.split('/')[-1]
    # Insert Local Video File Path
    clip = mp.VideoFileClip(path + ".mp4")

    # Insert Local Audio File Path

    clip.audio.write_audiofile("output.wav",codec='pcm_s16le')
    clip.close()

    sound = AudioSegment.from_file("output.wav", format="wav", channels=1)

    j = 0
    while (j + 5000 < len(sound)):
        j += 5000
        chunk = sound[j:(j + 5000)]
        with open(path+"-%s.wav" % j, "wb") as f:
            chunk.export(f, format="wav")
    try:
        os.remove(path + ".mp4")
    except:
        print(filename + ".mp4 could not be removed")

In [7]:
def download(link, path):
    audio_stream = YouTube(link).streams.filter(file_extension="mp4")
    audio_stream.first().download(output_path=path, filename="test.mp4")
    return audio_stream.first().title

In [22]:
def predict_genre(onclick):
    folder = BASE_PATH + "test/"
    if not os.path.exists(folder):
      os.makedirs(folder)
    else:
      for filename in os.listdir(folder):
        file_path = os.path.join(folder, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

    link = yt_input.value
    video_title = download_and_split(BASE_PATH + "test/", link)
    files = os.listdir(BASE_PATH + "test/")

    learn = load_learner(BASE_PATH + "multi_category_classifier.pkl")

    results = [0, 0, 0, 0]
    for file in files:
      temp_path = BASE_PATH + "test/" + file
      pred = int(learn.predict(Path(temp_path))[0])
      #print(pred)
      results[pred] += 1
    count, final_result = max((v, i) for i, v in enumerate(results))
    print(len(files))
    percentage = count / len(files) * 100
    output = "For " + video_title + "... "
    match final_result:
        case 0: output += "Classical "
        case 1: output += "Jazz "
        case 2: output += "Rock "
        case 3: output += "Hip Hop "
    output += "(Confidence: " + str(percentage) + "%)"
    output_text.value = output

In [23]:
# Define labels
subtitle, yt_text, output_text = widgets.Label(), widgets.Label(), widgets.Label()

# Title
subtitle.value = "Using a XResNet18 architecture and fastai's learning rate optimization (Smith, 2017), predict an audio clip's genre"

# YouTube entry
yt_text.value = "Link some YouTube audio"
yt_input = widgets.Text()

submission_btn = widgets.Button(description = 'Classify')
submission_btn.on_click(predict_genre)

VBox([widgets.Label("Music classifier:"), subtitle, yt_text, yt_input, submission_btn, output_text])

# Link: https://www.youtube.com/watch?v=0geT6FWeX8w